# Use options

Options provided for taskbook can be created via "options" classes.

There are two categories of options:

* **ExperimentOptions**:
    These options are used to set some parameters for the experiment. For example, the number of experiment shots (`counts`),
    the averaging mode (`averaging_mode`), the acquisition type (`acquisition_type`) and so on. 

* **TaskBookOptions**:
    These options are used to set some parameters for the taskbook. These include the settings for operating the taskbook, such as `run_until` which specifies the task the taskbook should stop at.
    In addition, options for the constituent tasks of the taskbook can also be set, by specifying the task name and the options for that task.



In [ ]:
from __future__ import annotations

from laboneq_applications.core.options import (
    BaseExperimentOptions,
    TaskBookOptions,
)
from laboneq_applications.workflow import task, taskbook

# Create a new experiment options class

It is recommended to create options classes always with default values. This way, options can be used without specifying them, and the default values will be used.

In [ ]:
class NewExperimentOptions(BaseExperimentOptions):
    operand: int = 1

In [ ]:
opt = NewExperimentOptions()

In [ ]:
opt

# Create a new taskbook options class

The taskbook options class is used to set the options for the taskbook. They must inherit from `TaskBookOptions`. 
The options for the constituent tasks of the taskbook can also be set by specifying the task name and the options class used for that task.
As in the case of the experiment options, it is recommended to create the taskbook options class with default values.

For instances, the following line sets the type `NewExperimentOptions` and default values `NewExperimentOptions()` for the options of task `add`:

`add: NewExperimentOptions = NewExperimentOptions()`

In [ ]:
class NewTaskBookOptions(TaskBookOptions):
    add: NewExperimentOptions = NewExperimentOptions()
    multiply: NewExperimentOptions = NewExperimentOptions()

In [ ]:
taskbook_opt = NewTaskBookOptions()

In [ ]:
taskbook_opt.add.operand = 2

# Use options in the taskbook

Define tasks and taskbook to be used

`mytaskbook` contains task `mytask` which takes in an argument and an options. 

To use `mytaskbook` with options, we must declare the type of options to be used, via the `options` attribute of the taskbook decorator.

Note how the options for each task are automatically set by the taskbook options.

In [ ]:
@task
def add(x, options: NewExperimentOptions):
    return x + options.operand


@task
def multiply(x, options: NewExperimentOptions):
    return x * options.operand

In [ ]:
@taskbook(options=NewTaskBookOptions)
def mytaskbook(options: NewTaskBookOptions | None = None):
    add(x=1)
    multiply(x=2)

Create an options object and pass it to the taskbook when running it.

In [ ]:
taskbook_opt = mytaskbook.options()

In [ ]:
taskbook_opt.add.operand = 3
taskbook_opt.multiply.operand = 3

In [ ]:
res = mytaskbook(options=taskbook_opt)
res.tasks[1].output

In [ ]:
taskbook_opt.run_until = "add"

In [ ]:
res = mytaskbook(options=taskbook_opt)
res.tasks  # only the first task is executed

Of course, the options can be created directly via the option class.

In [ ]:
taskbook_opt = NewTaskBookOptions()

# Disable options

For quick prototyping, it is possible to disable the automatic forwarding of `options` by the taskbook to its tasks by not setting the options attribute of the taskbook decorator. However, we can still pass options manually to each task as standard Python dictionaries.

In [ ]:
@task
def add(x, options: int):
    return x + options


@task
def multiply(x, options: int):
    return x * options


@taskbook
def mytaskbook(options: dict | None = None):
    add(x=1, options=options["operand1"])
    multiply(x=2, options=options["operand2"])

In [ ]:
mytaskbook(options={"operand1": 1, "operand2": 2})